In [1]:
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import pandas as pd


labels_df = pd.read_csv('./labels.csv')
# labels_df.rename(columns={'id': 'name'}, inplace=True)
# define location of dataset
folder = './train/'
photos, labels = list(), list()
# enumerate files in the directory
for file in listdir(folder):
    filename = file.rstrip('.jpg')
    # print(labels_df.loc[labels_df['id'] == filename, ['breed']])
	# load image
    photo = load_img(folder + file, target_size=(200, 200))
    # convert to numpy array
    photo = img_to_array(photo)
    # store
    photos.append(photo)
    labels.append(labels_df.loc[labels_df['id'] == file]['breed'])
# convert to a numpy arrays
photos = asarray(photos)
labels = asarray(labels)
print(photos.shape, labels.shape)
# save the reshaped photos
save('dogs_breeds_photos.npy', photos)
save('dogs_breeds_labels.npy', labels)

(10222, 200, 200, 3) (10222, 0)


In [2]:
# load and confirm the shape
from numpy import load
photos = load('dogs_breeds_photos.npy', allow_pickle=True)
labels = load('dogs_breeds_labels.npy', allow_pickle=True)
print(photos.shape, labels.shape)

(10222, 200, 200, 3) (10222, 0)


In [3]:
from os import makedirs

labels_df = pd.read_csv('./labels.csv')
labels_list = labels_df['breed'].values
# create directories
dataset_home = 'dataset_dog_breeds/'
subdirs = ['train/', 'test/']
for subdir in subdirs:
    # create label subdirectories
    for labldir in labels_list:
        labldir = labldir + '/'
        newdir = dataset_home + subdir + labldir
        makedirs(newdir, exist_ok=True)

In [4]:
from random import seed
from random import random
from shutil import copyfile

# seed random number generator
seed(1)
# define ratio of pictures to use for validation
val_ratio = 0.25
# copy training dataset images into subdirectories
src_directory = 'train/'
for file in listdir(src_directory):
    src = src_directory + '/' + file
    name = file.rstrip('.jpg')
    dst_dir = 'train/'
    if random() < val_ratio:
        dst_dir = 'test/'
    breed_dir = labels_df.loc[labels_df['id'] == name]['breed'].values + '/'
    # if file.startswith('cat'):
    #     dst = dataset_home + dst_dir + 'cats/'  + file
    #     copyfile(src, dst)
    # elif file.startswith('dog'):
    #     dst = dataset_home + dst_dir + 'dogs/'  + file
    #     copyfile(src, dst)
    dst = dataset_home + dst_dir + breed_dir[0]  + file
    copyfile(src, dst)

relu
pooling layers

In [10]:

# baseline model for the dogs vs cats dataset
import sys
from matplotlib import pyplot
# from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
 
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = 'wynik'
	pyplot.savefig(filename + '_plot.png')
	# pyplot.close()

In [35]:
from tensorflow.keras.layers import Dropout

# define cnn model
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.3))
    model.add(Dense(120, activation='softmax'))
    # compile model
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = define_model()

datagen = ImageDataGenerator(rescale=1.0/255.0)

# prepare iterators
train_it = datagen.flow_from_directory('dataset_dog_breeds/train/',
	class_mode='binary', batch_size=64, target_size=(200, 200))
test_it = datagen.flow_from_directory('dataset_dog_breeds/test/',
	class_mode='binary', batch_size=64, target_size=(200, 200))
# fit model
history = model.fit(train_it, steps_per_epoch=len(train_it),
	validation_data=test_it, validation_steps=len(test_it), epochs=20, verbose=0)
# evaluate model
_, acc = model.evaluate(test_it, steps=len(test_it), verbose=0)
print('> %.3f' % (acc * 100.0))
# learning curves
summarize_diagnostics(history)

Found 7598 images belonging to 120 classes.
Found 2624 images belonging to 120 classes.


ValueError: in user code:

    File "c:\users\ajozefowicz\pycharmprojects\racedetector\venv\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "c:\users\ajozefowicz\pycharmprojects\racedetector\venv\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\users\ajozefowicz\pycharmprojects\racedetector\venv\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "c:\users\ajozefowicz\pycharmprojects\racedetector\venv\lib\site-packages\keras\engine\training.py", line 809, in train_step
        loss = self.compiled_loss(
    File "c:\users\ajozefowicz\pycharmprojects\racedetector\venv\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "c:\users\ajozefowicz\py

In [6]:
# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = SGD(learning_rate=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

model2 = define_model()

datagen = ImageDataGenerator(rescale=1.0/255.0)

# prepare iterators
train_it = datagen.flow_from_directory('dataset_dog_breeds/train/',
	class_mode='binary', batch_size=64, target_size=(200, 200))
test_it = datagen.flow_from_directory('dataset_dog_breeds/test/',
	class_mode='binary', batch_size=64, target_size=(200, 200))
# fit model
history = model2.fit(train_it, steps_per_epoch=len(train_it),
	validation_data=test_it, validation_steps=len(test_it), epochs=20, verbose=0)
# evaluate model
_, acc = model2.evaluate(test_it, steps=len(test_it), verbose=0)
print('> %.3f' % (acc * 100.0))
# learning curves
summarize_diagnostics(history)

c:\users\ajozefowicz\pycharmprojects\racedetector\venv\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Found 7598 images belonging to 120 classes.
Found 2624 images belonging to 120 classes.


KeyboardInterrupt: 

In [33]:
import numpy as np

folder = './Fred/'
file = 'test.jpg'
photo = load_img(folder + file, target_size=(200, 200))
# convert to numpy array
photo = img_to_array(photo)
data = []
data.append(photo)
data = np.array(data)
model2.predict(data)

array([[nan]], dtype=float32)